In [ ]:
import numpy as np
import sys
sys.path.append('/home/codio/workspace/.guides/hf')
from helper import *

%matplotlib inline
print('You\'re running python %s' % sys.version.split(' ')[0])

In [ ]:
<h3>The <code>hashfeatures</code> and <code>name2features</code> Functions</h3>
<p>Below, the <code>hashfeatures</code> and <code>name2features</code> functions will take the plain text names and convert them to feature vectors so that you'll be able to work with the data effectively.  

In [ ]:
def hashfeatures(baby, B, FIX):
    """
    Input:
        baby : a string representing the baby's name to be hashed
        B: the number of dimensions to be in the feature vector
        FIX: the number of chunks to extract and hash from each string
    
    Output:
        v: a feature vector representing the input string
    """
    v = np.zeros(B)
    for m in range(FIX):
        featurestring = "prefix" + baby[:m]
        v[hash(featurestring) % B] = 1
        featurestring = "suffix" + baby[-1*m:]
        v[hash(featurestring) % B] = 1
    return v


In [ ]:
def name2features(filename, B=128, FIX=3, LoadFile=True):
    """
    Output:
        X : n feature vectors of dimension B, (nxB)
    """
    # read in baby names
    if LoadFile:
        with open(filename, 'r') as f:
            babynames = [x.rstrip() for x in f.readlines() if len(x) > 0]
    else:
        babynames = filename.split('\n')
    n = len(babynames)
    X = np.zeros((n, B))
    for i in range(n):
        X[i,:] = hashfeatures(babynames[i], B, FIX)
    return X

In [ ]:
<p>In the code cell above, <code>name2features</code> reads every name in the given file and converts it into a 128-dimensional feature vector by first assembling substrings (based on the parameter 'FIX'), then hashing these assembled substrings and modifying the feature vector index (the modulo of the number of dimensions) that corresponds to this hash value. </p> 

<p>Can you see how the feature vector for each name changes with different parameters? (Understanding how these features are constructed will help you later on in the challenge.)<br></p>

In [ ]:
<h3>The <code>genTrainFeatures</code> Function</h3>
<p>We have provided you with a python function <code>genTrainFeatures</code>, which transforms the names into features and loads them into memory. 

In [ ]:
def genTrainFeatures(dimension=128):
    """
    Input: 
        dimension: desired dimension of the features
    Output: 
        X: n feature vectors of dimensionality d (nxd)
        Y: n labels (-1 = girl, +1 = boy) (n)
    """
    
    # Load in the data
    Xgirls = name2features("girls.train", B=dimension)
    Xboys = name2features("boys.train", B=dimension)
    X = np.concatenate([Xgirls, Xboys])
    
    # Generate Labels
    Y = np.concatenate([-np.ones(len(Xgirls)), np.ones(len(Xboys))])
    
    # shuffle data into random order
    ii = np.random.permutation([i for i in range(len(Y))])
    
    return X[ii, :], Y[ii]

In [ ]:
<p>You can call the following command to return two vectors, one holding all the concatenated feature vectors and one holding the labels of all boys and girls names.</p>

In [ ]:
X, Y = genTrainFeatures(128)

In [2]:
%%HTML
<h2> The Na&iuml;ve Bayes Classifier </h2>

<p> The Na&iuml;ve Bayes classifier is a linear classifier based on Bayes Rule. The following cells will walk you through steps and ask you to finish the necessary functions in a pre-defined order. <strong>As a general rule, you should avoid tight loops at all costs.</strong></p>

In [1]:
%%HTML
<h3>Part One: Class Probability [Graded]</h3>
<i>
<p>Estimate the class probability $P(y)$ in 
<b><code>naivebayesPY</code></b>. This should return the probability that a sample in the training set is positive or negative, independent of its features.
</p>
</i>

In [ ]:
def naivebayesPY(X, Y):
    """
    naivebayesPY(Y) returns [pos,neg]

    Computation of P(Y)
    Input:
        X : n input vectors of d dimensions (nxd)
        Y : n labels (-1 or +1) (nx1)

    Output:
        pos: probability p(y=1)
        neg: probability p(y=-1)
    """
    
    # add one positive and negative example to avoid division by zero ("plus-one smoothing")
    Y = np.concatenate([Y, [-1,1]])
    n = len(Y)
    # YOUR CODE HERE
    
    Y[Y ==-1 ] = 0
    print(Y)
    print(sum(Y)) #601
    print(len(Y))
    pos = sum(Y)/len(Y)
    neg = 1-pos
    print('pos', pos)
    return [pos, neg]
    #print(sum(Y)) # equals zero so we know there are the same number of 1s as -1s
    #print(n) 1202
    
    
    #return None
    
    
    
    raise NotImplementedError()

pos, neg = naivebayesPY(X,Y)

In [ ]:
# The following tests will check that the probabilities returned by your function sum to 1 (test1) and return the correct probabilities for a given set of input vectors (tests 2-4)

# Check that probabilities sum to 1
def naivebayesPY_test1():
    pos, neg = naivebayesPY(X,Y)
    return np.linalg.norm(pos + neg - 1) < 1e-5

# Test the Naive Bayes PY function on a simple example
def naivebayesPY_test2():
    x = np.array([[0,1],[1,0]])
    y = np.array([-1,1])
    pos, neg = naivebayesPY(x,y)
    pos0, neg0 = .5, .5
    test = np.linalg.norm(pos - pos0) + np.linalg.norm(neg - neg0)
    return test < 1e-5

# Test the Naive Bayes PY function on another example
def naivebayesPY_test3():
        x = np.array([[0,1,1,0,1],
            [1,0,0,1,0],
            [1,1,1,1,0],
            [0,1,1,0,1],
            [1,0,1,0,0],
            [0,0,1,0,0],
            [1,1,1,0,1]])    
        y = np.array([1,-1, 1, 1,-1,-1, 1])
        pos, neg = naivebayesPY(x,y)
        pos0, neg0 = 5/9., 4/9.
        test = np.linalg.norm(pos - pos0) + np.linalg.norm(neg - neg0)
        return test < 1e-5

# Tests plus-one smoothing
def naivebayesPY_test4():
    x = np.array([[0,1,1,0,1],[1,0,0,1,0]])    
    y = np.array([1,1])
    pos, neg = naivebayesPY(x,y)
    pos0, neg0 = 3/4., 1/4.
    test = np.linalg.norm(pos - pos0) + np.linalg.norm(neg - neg0)
    return test < 1e-5    
        
    
runtest(naivebayesPY_test1, 'naivebayesPY_test1')
runtest(naivebayesPY_test2,'naivebayesPY_test2')
runtest(naivebayesPY_test3,'naivebayesPY_test3')
runtest(naivebayesPY_test4,'naivebayesPY_test4')

In [ ]:
<h3>Part Two: Conditional Probability [Graded]</h3>
<p>Estimate the conditional probabilities $P([\mathbf{x}]_{\alpha}|y)$ in 
<b><code>naivebayesPXY</code></b>. Notice that by construction, our features are binary categorical features. Use a <b>categorical</b> distribution as model and return the probability vectors for each feature being 1 given a class label.  Note that the result will be two vectors of length d (the number of features), where the values represent the probability that feature i is equal to 1.
</p> 

In [ ]:
def naivebayesPXY(X,Y):
    """
    naivebayesPXY(X, Y) returns [posprob,negprob]
    
    Input:
        X : n input vectors of d dimensions (nxd)
        Y : n labels (-1 or +1) (n)
    
    Output:
        posprob: probability vector of p(x_alpha = 1|y=1)  (d)
        negprob: probability vector of p(x_alpha = 1|y=-1) (d)
    """
    
    # add one positive and negative example to avoid division by zero ("plus-one smoothing")
    n, d = X.shape
    X = np.concatenate([X, np.ones((2,d)), np.zeros((2,d))])
    Y = np.concatenate([Y, [-1,1,-1,1]])
    n, d = X.shape
    
    # YOUR CODE HERE
    
    #take the Bayes algo iteratively on each element of a given dimension
    #find the mean of each of the 1204 values in n???
    #Remember each row is a unique item or instance
    #the 128 features, d, represent the hashed apsects of the pre and suffix of a given name
    #Remember we're mitigating the curse of dimentionality by assuming conditional independant features
    #this means we want to take Bayes on each row. Then take the product of each of the individual probs
    #meaning the probability that a baby name is male of female is the SUM of the probabilities of the each aspect (the Y vlue) being male or female??
    #we take the exponentiated sum of logariths of each y term
    #Note X[1,:] refers to all features (columns)
    #Note X[:,1] gives us all rows in the first column
    # Y has shape (1204,)
    # X has shape(1204, 128)
    
    
    #take the mean (probability)
    #test = X[0][X==1]
   # Xjprob = X[1:,]/len(d)
    Xjmean = np.mean(X,0)
    Xjmeanlog = np.log(Xjmean)
    
    
    #classConditional = 2
    
    #Xjmeanlogsum = np.sum(Xjmeanlog)
    
    
    #posprob= np.argmax(np.sum(np.log(classConditional), np.log(pos)))     #np.sum(np.log(pos))
    posprob = np.mean(X[Y==1],0)
    negprob = np.mean(X[Y==-1],0)
    
    

    return posprob, negprob
    #return [posprob, negprob]
    raise NotImplementedError()
    

posprob, negprob = naivebayesPXY(X,Y)

In [ ]:
# The following tests check that your implementation of naivebayesPXY returns the same posterior probabilities as the correct implementation, in the correct dimensions

# test a simple toy example with two points (one positive, one negative)
def naivebayesPXY_test1():
    x = np.array([[0,1],[1,0]])
    y = np.array([-1,1])
    pos, neg = naivebayesPXY(x,y)
    pos0, neg0 = naivebayesPXY_grader(x,y)
    test = np.linalg.norm(pos - pos0) + np.linalg.norm(neg - neg0)
    return test < 1e-5

# test the probabilities P(X|Y=+1)
def naivebayesPXY_test2():
    pos, neg = naivebayesPXY(X,Y)
    posprobXY, negprobXY = naivebayesPXY_grader(X, Y)
    test = np.linalg.norm(pos - posprobXY) 
    return test < 1e-5

# test the probabilities P(X|Y=-1)
def naivebayesPXY_test3():
    pos, neg = naivebayesPXY(X,Y)
    posprobXY, negprobXY = naivebayesPXY_grader(X, Y)
    test = np.linalg.norm(neg - negprobXY)
    return test < 1e-5


# Check that the dimensions of the posterior probabilities are correct
def naivebayesPXY_test4():
    pos, neg = naivebayesPXY(X,Y)
    posprobXY, negprobXY = naivebayesPXY_grader(X, Y)
    return pos.shape == posprobXY.shape and neg.shape == negprobXY.shape

runtest(naivebayesPXY_test1,'naivebayesPXY_test1')
runtest(naivebayesPXY_test2,'naivebayesPXY_test2')
runtest(naivebayesPXY_test3,'naivebayesPXY_test3')
runtest(naivebayesPXY_test4,'naivebayesPXY_test4')

In [ ]:
<h3>Part Three: Log Lokelihood [Graded]</h3>

<i>
<p>Calculate the log likelihood $\log P(\mathbf{x}|y)$ for each point in X_test in 
<b><code>loglikelihood</code></b> and label Y_test. Recall that the likelihood is given by the product of the conditional probabilities of each feature and that $\log(ab) = \log a + \log b$.
</p> 
<i>

In [ ]:
def loglikelihood(posprob, negprob, X_test, Y_test):
    """
    loglikelihood(posprob, negprob, X_test, Y_test) returns loglikelihood of each point in X_test
    
    Input:
        posprob: conditional probabilities for the positive class (d)
        negprob: conditional probabilities for the negative class (d)
        X_test : features (nxd)
        Y_test : labels (-1 or +1) (n)
    
    Output:
        loglikelihood of each point in X_test (n)
    """
    n, d = X_test.shape
    loglikelihood = np.zeros(n)
    
    # YOUR CODE HERE
    #posprob, negprob = naivebayesPXY(X,Y)
    plog = (X_test[Y_test==1]@np.log(posprob)) +((1- X_test[Y_test==1])@np.log(1-posprob))
    nlog = (X_test[Y_test==-1]@np.log(negprob)) + ((1-X_test[Y_test==-1])@np.log(1-negprob))
    #print('plog', plog)
    #print('nlog', nlog)
    
    #apply
    loglikelihood[Y_test==1] = plog
    loglikelihood[Y_test==-1] = nlog
    
    return loglikelihood
    raise NotImplementedError()
    

# compute the loglikelihood of the training set
posprob, negprob = naivebayesPXY(X,Y)
loglikelihood(posprob,negprob,X,Y) 

In [ ]:
# The following tests check that your implementation of loglikelihood returns the same values as the correct implementation for three different datasets

X, Y = genTrainFeatures(128)
posprobXY, negprobXY = naivebayesPXY_grader(X, Y)

# test if the log likelihood of the training data are all negative
def loglikelihood_testneg():
    ll=loglikelihood(posprob,negprob,X,Y);
    return all(ll<0)

# test if the log likelihood of the training data matches the solution
def loglikelihood_test0():
    ll=loglikelihood(posprob,negprob,X,Y);
    llgrader=loglikelihood_grader(posprob,negprob,X,Y);
    return np.linalg.norm(ll-llgrader)<1e-5

# test if the log likelihood of the training data matches the solution
# (positive points only)
def loglikelihood_test0a():
    ll=loglikelihood(posprob,negprob,X,Y);
    llgrader=loglikelihood_grader(posprob,negprob,X,Y);
    return np.linalg.norm(ll[Y==1]-llgrader[Y==1])<1e-5

# test if the log likelihood of the training data matches the solution
# (negative points only)
def loglikelihood_test0b():
    ll=loglikelihood(posprob,negprob,X,Y);
    llgrader=loglikelihood_grader(posprob,negprob,X,Y);
    return np.linalg.norm(ll[Y==-1]-llgrader[Y==-1])<1e-5


# little toy example with two data points (1 positive, 1 negative)
def loglikelihood_test1():
    x = np.array([[0,1],[1,0]])
    y = np.array([-1,1])
    posprobXY, negprobXY = naivebayesPXY_grader(X, Y)
    loglike = loglikelihood(posprobXY[:2], negprobXY[:2], x, y)
    loglike0 = loglikelihood_grader(posprobXY[:2], negprobXY[:2], x, y)
    test = np.linalg.norm(loglike - loglike0)
    return test < 1e-5

# little toy example with four data points (2 positive, 2 negative)
def loglikelihood_test2():
    x = np.array([[1,0,1,0,1,1], 
        [0,0,1,0,1,1], 
        [1,0,0,1,1,1], 
        [1,1,0,0,1,1]])
    y = np.array([-1,1,1,-1])
    posprobXY, negprobXY = naivebayesPXY_grader(X, Y)
    loglike = loglikelihood(posprobXY[:6], negprobXY[:6], x, y)
    loglike0 = loglikelihood_grader(posprobXY[:6], negprobXY[:6], x, y)
    test = np.linalg.norm(loglike - loglike0)
    return test < 1e-5


# one more toy example with 5 positive and 2 negative points
def loglikelihood_test3():
    x = np.array([[1,1,1,1,1,1], 
        [0,0,1,0,0,0], 
        [1,1,0,1,1,1], 
        [0,1,0,0,0,1], 
        [0,1,1,0,1,1], 
        [1,0,0,0,0,1], 
        [0,1,1,0,1,1]])
    y = np.array([1, 1, 1 ,1,-1,-1, 1])
    posprobXY, negprobXY = naivebayesPXY_grader(X, Y)
    loglike = loglikelihood(posprobXY[:6], negprobXY[:6], x, y)
    loglike0 = loglikelihood_grader(posprobXY[:6], negprobXY[:6], x, y)
    test = np.linalg.norm(loglike - loglike0)
    return test < 1e-5


runtest(loglikelihood_testneg, 'loglikelihood_testneg (all log likelihoods must be negative)')
runtest(loglikelihood_test0, 'loglikelihood_test0 (training data)')
runtest(loglikelihood_test0a, 'loglikelihood_test0a (positive points)')
runtest(loglikelihood_test0b, 'loglikelihood_test0b (negative points)')
runtest(loglikelihood_test1, 'loglikelihood_test1')
runtest(loglikelihood_test2, 'loglikelihood_test2')
runtest(loglikelihood_test3, 'loglikelihood_test3')

In [ ]:
<h3>Part Four: Naive Bayes Prediction [Graded]</h3>


<p>Observe that for a test point $\mathbf{x}_{test}$, we should classify it as positive if the log ratio $\log\left(\frac{P(y=1 | \mathbf{x} = \mathbf{x}_{test})}{P(y=-1|\mathbf{x} = \mathbf{x}_{test})}\right) > 0$ and negative otherwise. Implement the <b><code>naivebayes_pred</code></b> by first calculating the log ratio $\log\left(\frac{P(y=1 | \mathbf{x} = \mathbf{x}_{test})}{P(y=-1|\mathbf{x} = \mathbf{x}_{test})}\right)$ for each test point in $\mathbf{x}_{test}$ using Bayes' rule and predict the label of the test points by looking at the log ratio. When calculating the log likelihood, think carefully how you can use the fact $\log \left(\frac{a}{b}\right) = \log{a} - \log{b}$ to simplify your calculations.
</p>


In [ ]:
def naivebayes_pred(pos, neg, posprob, negprob, X_test):
    """
    naivebayes_pred(pos, neg, posprob, negprob, X_test) returns the prediction of each point in X_test
    
    Input:
        pos: class probability for the negative class
        neg: class probability for the positive class
        posprob: conditional probabilities for the positive class (d)
        negprob: conditional probabilities for the negative class (d)
        X_test : features (nxd)
    
    Output:
        prediction of each point in X_test (n)
    """
    n, d = X_test.shape
    
    # YOUR CODE HERE
    
    ones = np.ones(n)
    negones = -np.ones(n)
    
    boyslikelihood = loglikelihood(posprob, negprob,X_test, ones)
    girlslikelihood = loglikelihood(posprob, negprob,X_test, negones)
    
    
    #likelipred = boyslikelihood + np.log(pos) - girlslikelihood - np.log(neg)
    likelipred = boyslikelihood - girlslikelihood
    likelipred2 = np.where(likelipred > 0 , 1 , -1)
    return likelipred2

    raise NotImplementedError()

In [ ]:
# The following tests check that your implementation of naivebayes_pred returns only 1s and -1s (test 1), and that it returns the same predicted values as the correct implementation for three different datasets (tests 2-4)

X,Y = genTrainFeatures_grader(128)
posY, negY = naivebayesPY_grader(X, Y)

# check whether the predictions are +1 or neg 1
def naivebayes_pred_test1():
    preds = naivebayes_pred(posY, negY, posprobXY, negprobXY, X)
    return np.all(np.logical_or(preds == -1 , preds == 1))

def naivebayes_pred_test2():
    naivebayesPXY_grader(X, Y)
    x_test = np.array([[0,1],[1,0]])
    preds = naivebayes_pred_grader(posY, negY, posprobXY[:2], negprobXY[:2], x_test)
    student_preds = naivebayes_pred(posY, negY, posprobXY[:2], negprobXY[:2], x_test)
    acc = analyze_grader("acc", preds, student_preds)
    return np.abs(acc - 1) < 1e-5

def naivebayes_pred_test3():
    x_test = np.array([[1,0,1,0,1,1], 
        [0,0,1,0,1,1], 
        [1,0,0,1,1,1], 
        [1,1,0,0,1,1]])
    naivebayesPXY_grader(X, Y)
    preds = naivebayes_pred_grader(posY, negY, posprobXY[:6], negprobXY[:6], x_test)
    student_preds = naivebayes_pred(posY, negY, posprobXY[:6], negprobXY[:6], x_test)
    acc = analyze_grader("acc", preds, student_preds)
    return np.abs(acc - 1) < 1e-5

def naivebayes_pred_test4():
    x_test = np.array([[1,1,1,1,1,1], 
        [0,0,1,0,0,0], 
        [1,1,0,1,1,1], 
        [0,1,0,0,0,1], 
        [0,1,1,0,1,1], 
        [1,0,0,0,0,1], 
        [0,1,1,0,1,1]])
    naivebayesPXY_grader(X, Y)
    preds = naivebayes_pred_grader(posY, negY, posprobXY[:6], negprobXY[:6], x_test)
    student_preds = naivebayes_pred(posY, negY, posprobXY[:6], negprobXY[:6], x_test)
    acc = analyze_grader("acc", preds, student_preds)
    return np.abs(acc - 1) < 1e-5

runtest(naivebayes_pred_test1, 'naivebayes_pred_test1')
runtest(naivebayes_pred_test2, 'naivebayes_pred_test2')
runtest(naivebayes_pred_test3, 'naivebayes_pred_test3')
runtest(naivebayes_pred_test4, 'naivebayes_pred_test4')

In [ ]:
You can now test your code with the following interactive name classification script:

In [ ]:
DIMS = 128
print('Loading data ...')
X,Y = genTrainFeatures(DIMS)
print('Training classifier ...')
pos, neg = naivebayesPY(X, Y)
posprob, negprob = naivebayesPXY(X, Y)
error = np.mean(naivebayes_pred(pos, neg, posprob, negprob, X) != Y)
print('Training error: %.2f%%' % (100 * error))

while True:
    print('Please enter a baby name>')
    yourname = input()
    if len(yourname) < 1:
        break
    xtest = name2features(yourname,B=DIMS,LoadFile=False)
    pred = naivebayes_pred(pos, neg, posprob, negprob, xtest)
    if pred > 0:
        print("%s, I am sure you are a baby boy.\n" % yourname)
    else:
        print("%s, I am sure you are a baby girl.\n" % yourname)

In [ ]:
<h2> Challenge: Feature Extraction</h2>

<p>Let's test how well your Na&iuml;ve Bayes classifier performs on a secret test set. If you want to improve your classifier modify <code>name2features2</code> below.   The automatic reader will use your Python script to extract features and train your classifier on the same names training set by calling the function with only one argument--the name of a file containing a list of names.  The given implementation is the same as the given <code>name2features</code> above.
</p>
  

In [ ]:
def hashfeatures(baby, B, FIX):
    v = np.zeros(B)
    for m in range(FIX):
        featurestring = "prefix" + baby[:m]
        v[hash(featurestring) % B] = 1
        featurestring = "suffix" + baby[-1*m:]
        v[hash(featurestring) % B] = 1
    return v

def name2features2(filename, B=128, FIX=3, LoadFile=True):
    """
    Output:
        X : n feature vectors of dimension B, (nxB)
    """
    # read in baby names
    if LoadFile:
        with open(filename, 'r') as f:
            babynames = [x.rstrip() for x in f.readlines() if len(x) > 0]
    else:
        babynames = filename.split('\n')
    n = len(babynames)
    X = np.zeros((n, B))
    for i in range(n):
        X[i,:] = hashfeatures(babynames[i], B, FIX=5)
        
    # YOUR CODE HERE
    raise NotImplementedError()
    return X

In [ ]:
# Autograder test cell- competition

In [ ]:
(Hint: You should be able to get >80% accuracy just by changing some of the default hyperparameters in the function argument.  If you'd like to try something more sophisticated, you can add to `name2features2`)

<h4>Credits</h4>
 The name classification idea originates from <a href="http://nickm.com">Nick Montfort</a>.